In [5]:
import pandas as pd
import numpy as np

# Read data

XYtr = pd.read_csv('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/XYtr.csv')


XYtr1 = XYtr.copy() # from stackoverflow
XYtr1['cdate'] = pd.to_datetime(XYtr1['cdate']).values.astype(np.float64)/8.64e+13

Ytr1 = XYtr1['total'].copy()
Xtr1 = XYtr1[['X.sales', 'cdate', 'fee1', 'fee2']].copy()

Xtr1 = Xtr1.astype(np.float64)

Xtr1 = Xtr1.fillna(0)

pred = pd.read_csv('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/pred.csv')

Xte = pd.read_csv('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/Xte.csv')

Xte1 = Xte.copy()
Xte1['cdate'] = pd.to_datetime(Xte1['cdate']).values.astype(np.float64)/8.64e+13
Xte1 = Xte1[['X.sales', 'cdate', 'fee1', 'fee2']].copy()
Xte1 = Xte1.astype(np.float64)
Xte1 = Xte1.fillna(0)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

from sklearn.linear_model import SGDRegressor

model3 = SGDRegressor(loss = 'epsilon_insensitive', alpha = 0, epsilon = 0).fit(Xtr1, Ytr1)

In [34]:
import warnings
warnings.filterwarnings('ignore')
import imageio
import numpy as np

fp = open('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/Xtr_fi.csv', 'w')
fp.write('id,fi1,fi2,fi3,fi4,fi5,fi6,fi7\n')
for i in range(XYtr.shape[0]):
    id = XYtr.loc[i,'id']
    f = XYtr.loc[i,'id'] + XYtr.loc[i, 'ext']
    try:
        pic = imageio.imread('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/images/images/' + f)
        fi1 = pic.shape[0] # width
        fi2 = pic.shape[1] # height
        fi3 = np.mean(pic[:,:,:]) # mean intensity of the image, brightness
        fi4 = pic.min() # minimum is the darkest portion of the image
        fi5 = np.mean(pic[:,:,0]) # 
        fi6 = np.mean(pic[:,:,1])
        fi7 = np.mean(pic[:,:,2])
    
    except:
        fi1 = np.nan
        fi2 = np.nan
        fi3 = np.nan
        fi4 = np.nan
        fi5 = np.nan
        fi6 = np.nan
        fi7 = np.nan
        pass
    
    fp.write('%s,%f,%f,%f,%f,%f,%f,%f\n' % (id, fi1,fi2,fi3,fi4,fi5,fi6,fi7))

fp.close()

fp = open('data/Xte_fi.csv', 'w')
fp.write('id,fi1,fi2,fi3,fi4,fi5,fi6,fi7\n')
for i in range(Xte.shape[0]):
    id = XYtr.loc[i,'id']
    f = XYtr.loc[i,'id'] + XYtr.loc[i, 'ext']
    try:
        pic = imageio.imread('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/images/images/' + f)
        fi1 = pic.shape[0] # width
        fi2 = pic.shape[1] # height
        fi3 = np.mean(pic[:,:,:]) # mean intensity of the image, brightness
        fi4 = pic.min() # minimum is the darkest portion of the image
        fi5 = np.mean(pic[:,:,0]) 
        fi6 = np.mean(pic[:,:,1])
        fi7 = np.mean(pic[:,:,2])
    
    except:
        fi1 = np.nan
        fi2 = np.nan
        fi3 = np.nan
        fi4 = np.nan
        fi5 = np.nan
        fi6 = np.nan
        fi7 = np.nan
        pass
    
    fp.write('%s,%f,%f,%f,%f,%f,%f,%f\n' % (id, fi1,fi2,fi3,fi4,fi5,fi6,fi7))

fp.close()

FileNotFoundError: [Errno 2] No such file or directory: 'data/Xte_fi.csv'

In [67]:
import warnings
warnings.filterwarnings('ignore')
# Make corpus and vocab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

K = 10
XYtr['description'] = XYtr['description'].fillna("NAN")
Xte['description'] = Xte['description'].fillna("NAN")
corpus = list(XYtr['description'])+list(Xte['description'])
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(corpus)

# https://stackoverflow.com/questions/62777131/how-to-remove-frequent-infrequent-features-from-sklearn-countvectorizer 
# Code to remove top X% most frequent words (use this to remove stop words).

total_features = len(vectorizer.vocabulary_)
top_vect = CountVectorizer(max_features=int(total_features * 0.1))
top_bow = top_vect.fit_transform(corpus)

# Create a list of (term, frequency) tuples sorted by their frequency
sum_words = bow.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)

# Keep only the terms in a list
vocabulary, _ = zip(*words_freq[:int(total_features*0.9)])
vocabulary = list(vocabulary)

bottom_vect = CountVectorizer(vocabulary=vocabulary)
corpus = bottom_vect.fit_transform(corpus)

In [91]:
lda = LatentDirichletAllocation(n_components = K)
lda.fit(corpus)
topics = lda.transform(corpus)
N = XYtr.shape[0]
topics

array([[0.12627303, 0.00416746, 0.00416698, ..., 0.00416694, 0.72996326,
        0.00416825],
       [0.01428609, 0.01429223, 0.01428644, ..., 0.01428588, 0.87141051,
        0.01428645],
       [0.00357159, 0.0035716 , 0.00357219, ..., 0.00357145, 0.00357161,
        0.00357159],
       ...,
       [0.00303045, 0.00303046, 0.97272454, ..., 0.00303032, 0.00303046,
        0.00303044],
       [0.27370022, 0.00322636, 0.00322725, ..., 0.00322658, 0.00322649,
        0.00322632],
       [0.00357159, 0.0035716 , 0.00357219, ..., 0.00357145, 0.00357161,
        0.00357159]])

In [127]:
from sklearn.decomposition import NMF
model = NMF(n_components=K, init='random', random_state=0)
model.fit(corpus)
topics = model.transform(corpus)
topics  #https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

array([[5.41272357e-02, 2.53794763e-02, 1.10785018e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.14905026e-03, 0.00000000e+00, 9.49580740e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.36435598e-01, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.68938608e-02, 0.00000000e+00, 0.00000000e+00, ...,
        5.08972082e-03, 0.00000000e+00, 1.30880754e-04],
       [3.77500455e-02, 2.74156440e-02, 1.05799455e-02, ...,
        0.00000000e+00, 4.35445096e-02, 0.00000000e+00],
       [2.57978627e-04, 0.00000000e+00, 0.00000000e+00, ...,
        2.34265640e-01, 3.92715078e-06, 4.94409608e-05]])

In [123]:
# TruncSVD

#https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

model = TruncatedSVD(n_components=K, n_iter=7, random_state=0)
model.fit(corpus)
topics = model.transform(corpus)
topics  

array([[ 1.44514736,  0.2581813 ,  0.2390802 , ..., -0.10310143,
         0.62393678,  0.12994178],
       [ 0.31584444,  0.0292451 ,  0.0317513 , ...,  0.03485077,
         0.44780215, -0.35347414],
       [ 3.31738456, -3.94647718,  0.65413359, ...,  0.51851242,
         0.1945726 ,  0.21135198],
       ...,
       [ 3.82348818, -4.31512705, -0.58256104, ..., -0.55218988,
        -0.40816224, -0.20042469],
       [ 2.59864214,  0.43791885, -0.67200569, ..., -0.08676778,
         0.63878151, -0.16338286],
       [ 3.31417367, -3.92997903,  0.64487783, ...,  0.50242888,
         0.188201  ,  0.20401623]])

In [118]:
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html
# Kernel PCA(doesn't improve the score)
from sklearn.decomposition import KernelPCA
model = KernelPCA(n_components=K, kernel='poly')
model.fit(corpus)
topics = model.transform(corpus)
topics  

array([[-0.00573256, -0.01903385,  0.01716227, ...,  0.00071926,
        -0.00170657,  0.00036358],
       [-0.00874385, -0.03601306,  0.01522739, ...,  0.0037761 ,
         0.00077169, -0.00618773],
       [-0.00355072,  0.00543401, -0.05221697, ...,  0.01887486,
         0.00733082,  0.00163375],
       ...,
       [-0.00260077,  0.0138497 , -0.05800613, ..., -0.0106918 ,
        -0.00721132, -0.00782638],
       [-0.00274553, -0.00192695,  0.02068258, ..., -0.01231826,
         0.00362545, -0.01358288],
       [-0.00355052,  0.00541854, -0.05194489, ...,  0.01837211,
         0.00709648,  0.0016014 ]])

In [128]:
fp = open('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/XYtr_ft.csv', 'w')
fp.write('id')
for k in range(K):
    fp.write(',FT%04d' % k)

fp.write('\n')
for i in range(N):
    id = XYtr.loc[i,'id']
    fp.write('%s' % id)
    for k in range(K):
        fp.write(',%f' % topics[i, k])
    
    fp.write('\n')

fp.close()    

fp = open('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/Xte_ft.csv', 'w')
fp.write('id')
for k in range(K):
    fp.write(',FT%04d' % k)

fp.write('\n')
for i in range(N):
    id = Xte.loc[i,'id']
    fp.write('%s' % id)
    for k in range(K):
        fp.write(',%f' % topics[i + N, k])
    
    fp.write('\n')

fp.close()

In [129]:
Xtr_fi = pd.read_csv('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/Xtr_fi.csv')
Xtr_fi1 = Xtr_fi.copy()
Xtr_fi1.drop('id',1, inplace = True)
Xtr_fi1 = Xtr_fi1.fillna(0)

Xte_fi = pd.read_csv('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/Xte_fi.csv')
Xte_fi1 = Xte_fi.copy()
Xte_fi1.drop('id',1, inplace = True)
Xte_fi1 = Xte_fi1.fillna(0)

Xtr_ft = pd.read_csv('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/XYtr_ft.csv')
Xtr_ft1 = Xtr_ft.copy()
Xtr_ft1.drop('id',1, inplace = True)
Xtr_ft1 = Xtr_ft1.fillna(0)

Xte_ft = pd.read_csv('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/Xte_ft.csv')
Xte_ft1 = Xte_ft.copy()
Xte_ft1.drop('id',1, inplace = True)
Xte_ft1 = Xte_ft1.fillna(0)

In [130]:
Xtr5 = pd.concat([Xtr1, Xtr_fi1, Xtr_ft1], axis = 1)
Xte5 = pd.concat([Xte1, Xte_fi1, Xte_ft1], axis = 1)
Xte5 = Xte5.astype(np.float64)
Xte5 = Xte5.fillna(0)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Xtr5 = scaler.fit_transform(Xtr5)
Xte5 = scaler.transform(Xte5)

from sklearn.linear_model import SGDRegressor

model5 = SGDRegressor(loss = 'epsilon_insensitive', alpha = 0, epsilon = 0).fit(Xtr5, Ytr1)
pred5 = pred.copy()
pred5['total'] = model5.predict(Xte5)
pred5.to_csv('/Users/tanlu/Desktop/~/2021 Fall/Stat 440/project2/stat440-21-project2/pred5.csv', index = False)